In [2]:
import os
import pandas as pd

In [3]:
# df = pd.read_excel("/home/sakshmeno/Documents/GitHub/C-RNN-approach/Labels/Labelling_Prateek_Guillermo.xlsx")
df = pd.read_excel("/Users/saksh.menon/Documents/GitHub/C-RNN-approach/Labels/Labelling_Prateek_Guillermo.xlsx")

In [4]:
def code_preprocessing(file):
    with open(file) as dataset_obj:
        codeLines = dataset_obj.read()
    comment_lines = []
    raw_codeLines = codeLines.replace("\t","").split("\n")
    multi_line_flag = 0
    for line_number in range(len(raw_codeLines)):
        if multi_line_flag:
            if raw_codeLines[line_number].__contains__("*/"):
                multi_line_flag = 0
            comment_lines.append(line_number)
        elif raw_codeLines[line_number].__contains__("/*"):
            if raw_codeLines[line_number].startswith("/*") and not(raw_codeLines[line_number].__contains__("*/")):
                comment_lines.append(line_number)
                multi_line_flag = 1
            elif raw_codeLines[line_number].__contains__("/*") and not(raw_codeLines[line_number].startswith("/*")):
                if raw_codeLines[line_number].__contains__("*/"):
                    psuedo_multi_line_start = raw_codeLines[line_number].find("/*")
                    psuedo_multi_line_end = raw_codeLines[line_number].find("*/")
                    temporary_line = raw_codeLines[line_number][:psuedo_multi_line_start] + raw_codeLines[line_number][psuedo_multi_line_end+2:]
                    raw_codeLines[line_number] = temporary_line

        elif raw_codeLines[line_number].startswith("//"):
            comment_lines.append(line_number)
        elif raw_codeLines[line_number].__contains__("//"):
            comment_start = raw_codeLines[line_number].find('//')
            raw_codeLines[line_number] = raw_codeLines[line_number][:comment_start]
    
    

    comment_lines.reverse()
    for i in comment_lines:
        raw_codeLines.pop(i)

    def insert_space(string, index):
        string_copy = ""
        for i in range(len(string)):
            if i==(index):
                string_copy += " "
                string_copy += string[i]
                string_copy += " "
                continue
            string_copy += string[i]
        return string_copy

    def find_char_indices(input_string, char):
        indices = []
        replacement_token = 0
        for index, character in enumerate(input_string):
            if character == char:
                indices.append(index + 2*replacement_token)
                replacement_token+=1
        return indices

    def space_out(string, char):
        indices = find_char_indices(string, char)

        for i in indices:
            string = insert_space(string, i)
        return string

    for line_number in range(len(raw_codeLines)):
        placeHolder = raw_codeLines[line_number]
        placeHolder = space_out(placeHolder, ";")
        placeHolder = space_out(placeHolder, "(")
        placeHolder = space_out(placeHolder, ")")
        placeHolder = space_out(placeHolder, ",")
        if placeHolder.endswith(";"):
            raw_codeLines[line_number] = "<start> " + placeHolder.replace(";","<end>")
        elif not(placeHolder.endswith(";")):
            raw_codeLines[line_number] = "<start> " + placeHolder + " <end>"

    return raw_codeLines


In [5]:
def comment_finder(file):
    with open(file) as dataset_obj:
        codeLines = dataset_obj.read()
        comment_lines = []
        raw_codeLines = codeLines.replace("\t","").split("\n")
        multi_line_flag = 0
        for line_number in range(len(raw_codeLines)):
            if multi_line_flag:
                if raw_codeLines[line_number].__contains__("*/"):
                    multi_line_flag = 0
                comment_lines.append(line_number)
            elif raw_codeLines[line_number].__contains__("/*"):
                if raw_codeLines[line_number].startswith("/*") and not(raw_codeLines[line_number].__contains__("*/")):
                    comment_lines.append(line_number)
                    multi_line_flag = 1
                elif raw_codeLines[line_number].__contains__("/*") and not(raw_codeLines[line_number].startswith("/*")):
                    if raw_codeLines[line_number].__contains__("*/"):
                        psuedo_multi_line_start = raw_codeLines[line_number].find("/*")
                        psuedo_multi_line_end = raw_codeLines[line_number].find("*/")
                        temporary_line = raw_codeLines[line_number][:psuedo_multi_line_start] + raw_codeLines[line_number][psuedo_multi_line_end+2:]
                        raw_codeLines[line_number] = temporary_line

            elif raw_codeLines[line_number].startswith("//"):
                comment_lines.append(line_number)
            elif raw_codeLines[line_number].__contains__("//"):
                comment_start = raw_codeLines[line_number].find('//')
                raw_codeLines[line_number] = raw_codeLines[line_number][:comment_start]
    return comment_lines

In [6]:
file_start = {}
for i in range(len(df['File'])):
    if pd.isnull(df['File'][i]) == False:
        file_start[df['File'][i]] = i

BOTTOM_LINE = file_start['TOTAL NO. OF FILES']
file_list = list(file_start.keys())[:-1]
file_vulnerabilities = {}

for i in range(len(file_list)-1):
    vulnerable_lines=[]
    for j in range(file_start[file_list[i]],file_start[file_list[i+1]]-1):
        if (df['Comment'][j].split('|')[1].split()[0]) == "BRANCH:":
            if (pd.isna(df.at[j,'False Positive'])):
                vulnerable_lines.append(int(df['Comment'][j].split('|')[0].split()[1]))
        if (not(pd.isna(df.at[j,'Lines Missed']))):
            if (not(pd.isna(df.at[j,'Branch.1']))):
                line = df['Lines Missed'][j].split()
                if (not(line[0].isnumeric())):
                    vulnerable_lines.append(int(line[1].strip(":")))
                else:
                    vulnerable_lines.append(int(line[0].strip(":")))
    vulnerable_lines.sort()

    file_vulnerabilities[file_list[i]]=vulnerable_lines

file_list = file_list[:-1]

In [8]:
# os.chdir("/home/sakshmeno/Documents/GitHub/C-RNN-approach/dataset")
os.chdir("/Users/saksh.menon/Documents/GitHub/C-RNN-approach/dataset")
for file in file_list:
    comments = comment_finder(file)
    for vul_line in range(len(file_vulnerabilities[file])):
        for num in range(len(comments)):
            if num == len(comments)-1:
                (file_vulnerabilities[file][vul_line]) = ((file_vulnerabilities[file][vul_line])-(len(comments)))
            elif comments[num] < (file_vulnerabilities[file][vul_line]):
                None
            elif comments[num] > (file_vulnerabilities[file][vul_line]):
                (file_vulnerabilities[file][vul_line]) = ((file_vulnerabilities[file][vul_line])-(num+1))
                break


In [50]:
labeled_dataset = {'Lines':[], 'Label':[]}
labeled_dataset = pd.DataFrame(labeled_dataset)
labeled_dataset

for file in file_list:
    code = code_preprocessing(file)
    for line in range(len(code)):
        if (line) in file_vulnerabilities[file]:
            data = {'Lines': code[line], 'Label': 'Insecure'}
        else:
            data = {'Lines': code[line], 'Label': 'Secure'}
        labeled_dataset.loc[len(labeled_dataset)] = data

In [51]:
raw_code = list(labeled_dataset['Lines'])

In [52]:
raw_code #corpus

['<start>  <end>',
 '<start>  <end>',
 '<start> #include <Compiler.h> <end>',
 '<start> #include <TimeDelay.h> <end>',
 '<start> #include "HardwareProfile.h" <end>',
 '<start> #include "Peripherals/Nand.h" <end>',
 '<start> #include "Peripherals/Accel.h" <end>',
 '<start> #ifdef USE_GYRO <end>',
 '<start> #include "Peripherals/Gyro.h" <end>',
 '<start> #endif <end>',
 '<start> #include "Peripherals/Rtc.h" <end>',
 '<start> #include "USB/USB.h" <end>',
 '<start> #include "USB/usb_function_msd.h" <end>',
 '<start> #include "USB/usb_function_cdc.h" <end>',
 '<start> #include "MDD File System/FSIO.h" <end>',
 '<start> #include "Usb/USB_CDC_MSD.h" <end>',
 '<start> #include "Ftl/FsFtl.h" <end>',
 '<start> #include "Utils/FsUtils.h" <end>',
 '<start> #include "Utils/Fifo.h" <end>',
 '<start> #include "Utils/Util.h" <end>',
 '<start> #include "Peripherals/Analog.h" <end>',
 '<start> #include "Settings.h" <end>',
 '<start> #include "Logger.h" <end>',
 '<start>  <end>',
 '<start> #include "AX3 

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

from keras.models import Sequential
from keras.layers import Dense, SimpleRNN

In [89]:
vectorizer = TfidfVectorizer()
transformed_output = vectorizer.fit_transform(raw_code)
print(len(vectorizer.vocabulary_.keys()))

2020


In [72]:
labeled_dataset.shape

(5205, 2)

In [54]:
all_feature_names = vectorizer.get_feature_names_out()

for word in all_feature_names:
    
    #let's get the index in the vocabulary
    indx = vectorizer.vocabulary_.get(word)
    
    #get the score
    idf_score = vectorizer.idf_[indx]
    
    print(f"{word} : {idf_score}")

00 : 8.864419904994566
0000 : 8.864419904994566
0000001 : 8.864419904994566
00_resource_generation : 8.864419904994566
01 : 8.864419904994566
02x : 7.611656936499197
08x : 7.254981992560465
0b00001001 : 8.864419904994566
0d100 : 8.864419904994566
0x : 7.254981992560465
0x0 : 5.446693221381199
0x00 : 7.07266043576651
0x0000 : 7.611656936499197
0x0000000f : 8.4589547968864
0x001f : 8.864419904994566
0x00ff : 7.15967181275614
0x01 : 8.171272724434619
0x02 : 8.864419904994566
0x05 : 8.864419904994566
0x06 : 8.864419904994566
0x0640 : 8.864419904994566
0x07e0 : 8.864419904994566
0x07ff : 8.864419904994566
0x0b : 8.864419904994566
0x0d : 8.864419904994566
0x0e : 8.864419904994566
0x0f : 6.724353741498295
0x1 : 7.360342508218292
0x10 : 8.4589547968864
0x10000ul : 8.864419904994566
0x11 : 8.864419904994566
0x113 : 8.864419904994566
0x12 : 8.864419904994566
0x1234 : 8.864419904994566
0x12345678 : 8.864419904994566
0x13 : 8.864419904994566
0x14 : 8.864419904994566
0x15 : 8.864419904994566
0x16 :

In [57]:
labeled_dataset['Label']=labeled_dataset['Label'].map({"Secure" : 0, "Insecure":1})
x_train, x_test, y_train, y_test = train_test_split(labeled_dataset['Lines'], labeled_dataset['Label'], test_size=0.2, random_state=32, stratify=labeled_dataset['Label'])
x_train, x_test, y_train, y_test

In [154]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, 
                        activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

demo_model = create_RNN(2, 1, (None,1805), activation=['linear', 'linear'])

clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('RNN', demo_model)         
])

#2. fit with X_train and y_train
clf.fit(x_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(x_test)


#4. print the classfication report
print(classification_report(y_test, y_pred))

NameError: name 'Sequential' is not defined

In [106]:
x_vector = labeled_dataset['Lines']
y_vector = labeled_dataset['Label']

vectorizer = TfidfVectorizer().fit(x_vector)
x_vector = vectorizer.transform(x_vector).toarray()
y_vector = vectorizer.transform(y_vector).toarray()
x_vector = x_vector[:, :, None]
y_vector = y_vector[:, :, None]


AttributeError: 'int' object has no attribute 'lower'

In [105]:
x_train = x_train[:, :, None]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed